# Tool Use in Agentic AI Systems

 ## 1.  Introduction
This notebook demonstrates **tool use** as a core building block of agentic AI systems using a retail dataset example.

This agentic workflow can carry out various tasks related to answering questions about a retail dataset. We will give it natural language instructions - like “What was the largest invoice in December 2010” or “What was the total value of invoice 536365?” - and see how it selects the right tools and completes the task for us.

## 2. Setup: Initialize environment and client

In this step, you import the key libraries that will support the workflow:  

- **`json`**: Provides functions to read and write JSON, useful for handling structured responses returned by the LLM.  
- **`utils`**: A custom helper module provided for this lab. It includes utility functions to work with the dataset, generate charts, and display results in a clean, readable format.
- **`openpyxl`**: A library used to read and write Excel files in Python. It enables loading the Online Retail dataset from an `.xlsx` file into a pandas DataFrame.


In [15]:
import sys
sys.path.append("../../agentic-ai")

In [18]:

# Standard library imports
import os 
import pandas as pd
from utils import utils
import json 

## 3. Demonstrating Tool Use

To demonstrate tool use, we follow the following steps sequentially:

- **Answering without Tools** - Ask questions about the dataset without the agent having any context. Obviously, we do not expect to get a valid response to our query.
- **Create required tools** - Define the tools that are need to answer our queries about the dataset. We create tools to lookup a specific invoice, aggregate revenues over a specified date range, summarize transactions for a specific customer and compare revenues between two countries.
- **Force tool use** - We force the agent to answer queries from a specific tool.
- **Tool selection by Agent** - We allow the agent to autonomously decide which tool is required to be executed to answer our query.
- **Tool failure** - When the agent decides to execute a specific tool, but does not find answers to the query from the database. 
- **No tool call** -   Queries that cannot be answered by the agent based on the available tools. Here, the agent decides not to execute a tool.

In [ ]:
# Read the retail dataset (downloaded from the UCI repository)
# url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
path = os.path.join("..", "data", "online_retail.xlsx")

# df = pd.read_excel(url)
df = pd.read_excel(path)

In [20]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [21]:
# Calculate revenue for each transaction
df['Revenue'] = df['Quantity'] * df['UnitPrice']

### 3.1 Answering a Question Without Tools

In [22]:
question = (
    "What were the average daily sales for Store 25 "
    "during promotional days in July 2014?"
)

prompt = f"""
You are a helpful assistant.

Answer the following question as best as you can.

Question:
"{question}"

If you do not know the exact answer, explain your reasoning.
"""

response = utils.get_response(model="gpt-4.1", prompt=prompt)
utils.print_html(response, title="LLM Answer Without Tools")


### 3.2 Create required tools 

In [28]:

def invoice_lookup(df, invoice_no: int):
    """
    Look up details for a specific invoice number.
    Args:
        df (pd.DataFrame): The retail dataset.
        invoice_no (int): The invoice number to look up.
        
    Returns:
        dict: A dictionary containing total revenue, total items, and number of unique products for the given invoice number.
    """
    subset = df[df["InvoiceNo"] == invoice_no]

    if subset.empty:
        return None

    return {
        "total_revenue": subset["Revenue"].sum(),
        "total_items": subset["Quantity"].sum(),
        "num_products": subset["StockCode"].nunique()
    }


In [29]:
def revenue_aggregate(
    df,
    start_date: str,
    end_date: str,
    aggregation: str = "sum",
    country: str | None = None,
):
    """Calculate aggregated revenue over a date range, optionally filtered by country.
    
    Args:
        df (pd.DataFrame): The retail dataset.
        start_date (str): The start date in "YYYY-MM-DD" format.
        end_date (str): The end date in "YYYY-MM-DD" format.
        aggregation (str): The type of aggregation ("sum", "mean", "median").
        country (str, optional): If provided, filter by this country.
        
    Returns:
        float: The aggregated revenue, or None if no data matches the criteria.

    """
    subset = df[
        (df["InvoiceDate"] >= start_date) &
        (df["InvoiceDate"] <= end_date)
    ]

    if country:
        subset = subset[subset["Country"] == country]

    if subset.empty:
        return None

    if aggregation == "sum":
        return subset["Revenue"].sum()
    if aggregation == "mean":
        return subset["Revenue"].mean()
    if aggregation == "median":
        return subset["Revenue"].median()

    raise ValueError("Unsupported aggregation")

In [30]:
def customer_summary(df, customer_id: float):
    """
    Generate a summary of a customer's purchasing behavior.
    Args:
        df (pd.DataFrame): The retail dataset.
        customer_id (float): The customer ID to summarize.
    Returns:
        dict: A dictionary containing total spent, number of invoices, and number of unique products for the given customer ID.
    """
    subset = df[df["CustomerID"] == customer_id]

    if subset.empty:
        return None

    return {
        "total_spent": subset["Revenue"].sum(),
        "num_invoices": subset["InvoiceNo"].nunique(),
        "num_products": subset["StockCode"].nunique()
    }


In [31]:
def country_compare(
    df,
    country_a: str,
    country_b: str,
    start_date: str,
    end_date: str,
):
    """ Compare total revenue between two countries over a specified date range.
    Args:
        df (pd.DataFrame): The retail dataset.
        country_a (str): The first country to compare.
        country_b (str): The second country to compare.
        start_date (str): The start date in "YYYY-MM-DD" format.
        end_date (str): The end date in "YYYY-MM-DD" format.
    Returns:
        dict: A dictionary with total revenue for each country.
    """
    subset = df[
        (df["InvoiceDate"] >= start_date) &
        (df["InvoiceDate"] <= end_date)
    ]

    a_val = subset[subset["Country"] == country_a]["Revenue"].sum()
    b_val = subset[subset["Country"] == country_b]["Revenue"].sum()

    return {
        country_a: a_val,
        country_b: b_val
    }


In [32]:
def execute_tool(tool_name: str, arguments: dict, df):
    """
    Execute a specified tool with given arguments on the dataset.
    Args:
        tool_name (str): The name of the tool to execute.
        arguments (dict): The arguments to pass to the tool.
        df (pd.DataFrame): The retail dataset.
    
    Returns:
        The result of executing the tool.
    """
    if tool_name == "invoice_lookup":
        return invoice_lookup(df, **arguments)

    if tool_name == "revenue_aggregate":
        return revenue_aggregate(df, **arguments)

    if tool_name == "customer_summary":
        return customer_summary(df, **arguments)

    if tool_name == "country_compare":
        return country_compare(df, **arguments)

    raise ValueError("Unknown tool")


### 3.3 Forcing Tool Use

In [33]:
question = "What was the total value of invoice 536365?"

In [34]:
# Force tool use

value = execute_tool(
    tool_name="invoice_lookup",
    arguments={"invoice_no": 536365},
    df=df
)


utils.print_html(value, title="Invoice Lookup Result")


### 3.4 Tool Selection by Agent

In [35]:
TOOL_SELECTION_PROMPT = """
You are an AI agent with access to the following tools:

1. invoice_lookup
   Retrieve summary information for a specific invoice.
   {{
      "invoice_no": int
   }}

2. revenue_aggregate
   Compute revenue statistics over a date range.
   {{
      "start_date": "YYYY-MM-DD",
      "end_date": "YYYY-MM-DD",
      "aggregation": "sum" | "mean",
      "country": string (optional)
   }}
   

3. customer_summary
   Retrieve summary statistics for a specific customer.
   {{
      "customer_id": float
   }}


4. country_compare
   Compare revenue between two countries over a time period.
   country_compare:
   {{
      "country_a": string,
      "country_b": string,
      "start_date": "YYYY-MM-DD",
      "end_date": "YYYY-MM-DD"
   }}
   

IMPORTANT:
- Use EXACT argument names as shown above.
- Do not rename keys.
- Use correct types.


User question:
"{question}"

Decide:
- whether a tool is required
- which tool is most appropriate
- the arguments to pass

Respond with valid JSON only:
{{
  "use_tool": true or false,
  "tool_name": "invoice_lookup" | "revenue_aggregate" | "customer_summary" | "country_compare" | null,
  "arguments": object | null
}}
"""


In [36]:
# question = "What was average revenue in Germany in Q1 2011?"
question = "What was the largest invoice in December 2010?"

In [37]:
decision = json.loads(
    utils.get_response(
        model="gpt-4.1-mini",
        prompt=TOOL_SELECTION_PROMPT.format(question=question),
    )
)

utils.print_html(decision, title="Tool Selection Decision")


In [38]:
tool_output = None

if decision.get("use_tool") and decision.get("tool_name"):

    tool_name = decision["tool_name"]
    arguments = decision.get("arguments", {})

    try:
        tool_output = execute_tool(
            tool_name=tool_name,
            arguments=arguments,
            df=df
        )
    except TypeError as e:
        print("Argument mismatch:", e)
        print("Arguments received:", arguments)
        tool_output = None

utils.print_html(tool_output, title="Tool Output")


### 3.5 Tool Failure Case

In [42]:
failure_question = "What was the total value of invoice 999999?"

In [43]:
# Get tool decision
failure_decision = json.loads(
    utils.get_response(
        model="gpt-4.1-mini",
        prompt=TOOL_SELECTION_PROMPT.format(question=failure_question),
    )
)

utils.print_html(failure_decision, title="Tool Selection (Failure Case)")

# Execute tool
failure_output = None

if failure_decision.get("use_tool") and failure_decision.get("tool_name"):

    tool_name = failure_decision["tool_name"]
    arguments = failure_decision.get("arguments", {})

    try:
        failure_output = execute_tool(tool_name, arguments, df)
    except Exception as e:
        print("Execution error:", e)
        failure_output = None

utils.print_html(failure_output, title="Tool Output (Failure Case)")


### 3.6 When No Tool Is Needed

In [44]:

reasoning_question = "Why do promotional discounts usually increase retail sales?"

reasoning_answer = utils.get_response(
    model="gpt-4.1",
    prompt=f"Answer clearly: {reasoning_question}"
)

utils.print_html(reasoning_answer, title="Final Answer (No Tool Used)")


## 4. Final Takeways

In this notebook we introduced:

- Tool contracts as capability interfaces
- Structured decision-making before execution
- Integration of tool outputs into reasoning
- Failure handling and conditional tool use


These ideas form the foundation formore advanced concepts in Agentic AI such as Multi-step planning, Memory-augmented agents and Reflection over tool choice
